In [1]:
!pip install tqdm
!pip install -U -q PyDrive
!pip install keras
!apt-get -qq install -y graphviz && pip install -q pydot

    100% |████████████████████████████████| 51kB 3.4MB/s 
Selecting previously unselected package fontconfig.
(Reading database ... 18298 files and directories currently installed.)
Preparing to unpack .../00-fontconfig_2.11.94-0ubuntu2_amd64.deb ...
Unpacking fontconfig (2.11.94-0ubuntu2) ...
Selecting previously unselected package libjbig0:amd64.
Preparing to unpack .../01-libjbig0_2.1-3.1_amd64.deb ...
Unpacking libjbig0:amd64 (2.1-3.1) ...
Selecting previously unselected package libcdt5.
Preparing to unpack .../02-libcdt5_2.38.0-16ubuntu2_amd64.deb ...
Unpacking libcdt5 (2.38.0-16ubuntu2) ...
Selecting previously unselected package libcgraph6.
Preparing to unpack .../03-libcgraph6_2.38.0-16ubuntu2_amd64.deb ...
Unpacking libcgraph6 (2.38.0-16ubuntu2) ...
Selecting previously unselected package libtiff5:amd64.
Preparing to unpack .../04-libtiff5_4.0.8-5ubuntu0.1_amd64.deb ...
Unpacking libtiff5:amd64 (4.0.8-5ubuntu0.1) ...
Selecting previously unselected package libwebp6:amd64.
Prep

Selecting previously unselected package libxt6:amd64.
Preparing to unpack .../24-libxt6_1%3a1.1.5-1_amd64.deb ...
Unpacking libxt6:amd64 (1:1.1.5-1) ...
Selecting previously unselected package libxmu6:amd64.
Preparing to unpack .../25-libxmu6_2%3a1.1.2-2_amd64.deb ...
Unpacking libxmu6:amd64 (2:1.1.2-2) ...
Selecting previously unselected package libxaw7:amd64.
Preparing to unpack .../26-libxaw7_2%3a1.0.13-1_amd64.deb ...
Unpacking libxaw7:amd64 (2:1.0.13-1) ...
Selecting previously unselected package graphviz.
Preparing to unpack .../27-graphviz_2.38.0-16ubuntu2_amd64.deb ...
Unpacking graphviz (2.38.0-16ubuntu2) ...
Setting up libpathplan4 (2.38.0-16ubuntu2) ...
Setting up libxcb-render0:amd64 (1.12-1ubuntu1) ...
Setting up libjbig0:amd64 (2.1-3.1) ...
Setting up libdatrie1:amd64 (0.2.10-5) ...
Setting up libtiff5:amd64 (4.0.8-5ubuntu0.1) ...
Setting up libgraphite2-3:amd64 (1.3.10-2) ...
Setting up libpixman-1-0:amd64 (0.34.0-1) ...
Processing triggers for libc-bin (2.26-0ubuntu2.1)

In [1]:
! git clone --recursive https://github.com/leftthomas/ImageDeblurring.git
!ls -l 

Cloning into 'ImageDeblurring'...
remote: Counting objects: 855, done.
remote: Total 855 (delta 0), reused 0 (delta 0), pack-reused 855
Receiving objects: 100% (855/855), 17.12 MiB | 48.83 MiB/s, done.
Resolving deltas: 100% (79/79), done.
total 8
drwxr-xr-x 1 root root 4096 May 21 17:02 datalab
drwxr-xr-x 4 root root 4096 Jun  6 04:10 ImageDeblurring


In [2]:
!ls -l /content

total 8
drwxr-xr-x 1 root root 4096 May 21 17:02 datalab
drwxr-xr-x 4 root root 4096 Jun  6 04:10 ImageDeblurring


In [3]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

Using TensorFlow backend.


In [0]:
from PIL import Image
import os, sys
import cv2
import numpy as np
pic_num = 301


        
dir = os.chdir('/content/ImageDeblurring/data/test/')

if not os.path.exists('/content/ImageDeblurring/data/testdata'):
        os.makedirs('/content/ImageDeblurring/data/testdata')
        
for file in os.listdir(dir):
    img = cv2.imread('/content/ImageDeblurring/data/test/'+file)
    resized_image = cv2.resize(img, (100, 100))
    cv2.imwrite("/content/ImageDeblurring/data/testdata/"+ file,resized_image)
    #pic_num = pic_num + 1
    #cv.imshow('exampleshq', thumbnail)
    #cv.waitKey(0)
    cv2.destroyAllWindows()
    os.chdir('/content/')

In [0]:
#!ls -l /content/ImageDeblurring/data/small/testdata
#os.chdir('/content')
#!pwd
#!mkdir -p result/test
!mkdir -p result/finally
#!mkdir weight
#!mkdir -p result/interim
#!ls -l /content/weight/

In [0]:
import glob as gb
import os

import h5py
import numpy as np
from PIL import Image
from tqdm import tqdm


# normalization x to [-1,1]
def normalization(x):
    return x / 127.5 - 1


# according the image path to read the image and covert it
# to the given size, then slice it, finally return the full and blur images
def format_image(image_path, size):
    image = Image.open(image_path)
    # slice image into full and blur images
    image_full = image.crop((0, 0, image.size[0] / 2, image.size[1]))
    # Note the full image in left, the blur image in right
    image_blur = image.crop((image.size[0] / 2, 0, image.size[0], image.size[1]))

    # image_full.show()
    # image_blur.show()

    image_full = image_full.resize((size, size), Image.ANTIALIAS)
    image_blur = image_blur.resize((size, size), Image.ANTIALIAS)

    # return the numpy arrays
    return np.array(image_full), np.array(image_blur)


# convert images to hdf5 data
def build_hdf5(jpeg_dir, size=100):
    # put data in HDF5
    hdf5_file = os.path.join('/content/ImageDeblurring/data', 'data.h5')
    with h5py.File(hdf5_file, 'w') as f:

        for data_type in tqdm(['traindata', 'testdata'], desc='create HDF5 dataset from images'):
            data_path = jpeg_dir + '/%s/*.jpg' % data_type
            images_path = gb.glob(data_path)
            # print(images_path)
            data_full = []
            data_blur = []
            for image_path in images_path:
                image_full, image_blur = format_image(image_path, size)
                data_full.append(image_full)
                data_blur.append(image_blur)

            # print(len(data_full))
            # print(len(data_blur))
            f.create_dataset('%s_data_full' % data_type, data=data_full)
            f.create_dataset('%s_data_blur' % data_type, data=data_blur)


# load data by data type
def load_data(data_type):
    with h5py.File('/content/ImageDeblurring/data/data.h5', 'r') as f:
        data_full = f['%s_data_full' % data_type][:].astype(np.float32)
        data_full = normalization(data_full)

        data_blur = f['%s_data_blur' % data_type][:].astype(np.float32)
        data_blur = normalization(data_blur)

        return data_full, data_blur


def generate_image(full, blur, generated, path, epoch=None, index=None):
    full = full * 127.5 + 127.5
    blur = blur * 127.5 + 127.5
    generated = generated * 127.5 + 127.5
    for i in range(generated.shape[0]):
        image_full = full[i, :, :, :]
        image_blur = blur[i, :, :, :]
        image_generated = generated[i, :, :, :]
        image = np.concatenate((image_full, image_blur, image_generated), axis=1)
        if (epoch is not None) and (index is not None):
            Image.fromarray(image.astype(np.uint8)).save(path + str(epoch + 1) + '_' + str(index + 1) + '.png')
        else:
            Image.fromarray(image.astype(np.uint8)).save(path + str(i) + '.png')


#if __name__ == '__main__':
 #   format_image('/content/ImageDeblurring/data/small/testdata/301.jpg', size=100)
 #   build_hdf5('/content/ImageDeblurring/data/small')
 #   img_full, img_blur = load_data('traindata')
 #   print(img_full, '\n', len(img_blur))


In [5]:
import keras.backend as K
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.models import Model

#import data_utils

# Note the image_shape must be multiple of patch_shape
image_shape = (100, 100, 3)
K_1 = 145
K_2 = 170


def l1_loss(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true))


def perceptual_loss(y_true, y_pred):
    vgg = VGG16(include_top=False, weights='imagenet', input_shape=image_shape)
    loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
    # let the loss model can't be trained
    loss_model.trainable = False
    # loss_model.summary()
    return K.mean(K.square(loss_model(y_true) - loss_model(y_pred)))


def generator_loss(y_true, y_pred):
    return K_1 * perceptual_loss(y_true, y_pred) + K_2 * l1_loss(y_true, y_pred)


def adversarial_loss(y_true, y_pred):
    return -K.log(y_pred)


if __name__ == '__main__':
    a, b = format_image('/content/ImageDeblurring/data/small/testdata/301.jpg', size=100)
    print(l1_loss(a.astype(np.float32), b.astype(np.float32)))


Using TensorFlow backend.


Tensor("Mean:0", shape=(), dtype=float32)


In [6]:
from keras.layers import Input, concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Convolution2D
from keras.layers.core import Dropout, Dense, Flatten, Lambda
from keras.layers.merge import Average
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.utils.vis_utils import plot_model

# the paper defined hyper-parameter:chr
channel_rate = 25
# Note the image_shape must be multiple of patch_shape
image_shape = (100, 100, 3)
patch_shape = (channel_rate, channel_rate, 3)


# Dense Block
def dense_block(inputs, dilation_factor=None):
    x = LeakyReLU(alpha=0.2)(inputs)
    x = Convolution2D(filters=4 * channel_rate, kernel_size=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    # the 3 Ã— 3 convolutions along the dense field are alternated between â€˜spatialâ€™ convolution
    # and â€˜dilatedâ€™ convolution with linearly increasing dilation factor
    if dilation_factor is not None:
        x = Convolution2D(filters=channel_rate, kernel_size=(3, 3), padding='same',
                          dilation_rate=dilation_factor)(x)
    else:
        x = Convolution2D(filters=channel_rate, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    # add Gaussian noise
    x = Dropout(rate=0.5)(x)
    return x


def generator_model():
    # Input Image, Note the shape is variable
    inputs = Input(shape=(None, None, 3))
    # The Head
    h = Convolution2D(filters=4 * channel_rate, kernel_size=(3, 3), padding='same')(inputs)

    # The Dense Field
    d_1 = dense_block(inputs=h)
    x = concatenate([h, d_1])
    # the paper used dilated convolution at every even numbered layer within the dense field
    d_2 = dense_block(inputs=x, dilation_factor=(1, 1))
    x = concatenate([x, d_2])
    d_3 = dense_block(inputs=x)
    x = concatenate([x, d_3])
    d_4 = dense_block(inputs=x, dilation_factor=(2, 2))
    x = concatenate([x, d_4])
    d_5 = dense_block(inputs=x)
    x = concatenate([x, d_5])
    d_6 = dense_block(inputs=x, dilation_factor=(3, 3))
    x = concatenate([x, d_6])
    d_7 = dense_block(inputs=x)
    x = concatenate([x, d_7])
    d_8 = dense_block(inputs=x, dilation_factor=(2, 2))
    x = concatenate([x, d_8])
    d_9 = dense_block(inputs=x)
    x = concatenate([x, d_9])
    d_10 = dense_block(inputs=x, dilation_factor=(1, 1))
    # The Tail
    x = LeakyReLU(alpha=0.2)(d_10)
    x = Convolution2D(filters=4 * channel_rate, kernel_size=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)

    # The Global Skip Connection
    x = concatenate([h, x])
    x = Convolution2D(filters=channel_rate, kernel_size=(3, 3), padding='same')(x)
    # PReLU can't be used, because it is connected with the input shape
    # x = PReLU()(x)
    x = LeakyReLU(alpha=0.2)(x)

    # Output Image
    outputs = Convolution2D(filters=3, kernel_size=(3, 3), padding='same', activation='tanh')(x)
    model = Model(inputs=inputs, outputs=outputs, name='Generator')
    return model


def discriminator_model():
    # PatchGAN
    inputs = Input(shape=patch_shape)
    x = Convolution2D(filters=channel_rate, kernel_size=(3, 3), strides=(2, 2), padding="same")(inputs)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Convolution2D(filters=2 * channel_rate, kernel_size=(3, 3), strides=(2, 2), padding="same")(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Convolution2D(filters=4 * channel_rate, kernel_size=(3, 3), strides=(2, 2), padding="same")(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Convolution2D(filters=4 * channel_rate, kernel_size=(3, 3), strides=(2, 2), padding="same")(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Flatten()(x)
    outputs = Dense(units=1, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=outputs, name='PatchGAN')
    # model.summary()

    # discriminator
    inputs = Input(shape=image_shape)

    list_row_idx = [(i * channel_rate, (i + 1) * channel_rate) for i in
                    range(int(image_shape[0] / patch_shape[0]))]
    list_col_idx = [(i * channel_rate, (i + 1) * channel_rate) for i in
                    range(int(image_shape[1] / patch_shape[1]))]

    list_patch = []
    for row_idx in list_row_idx:
        for col_idx in list_col_idx:
            x_patch = Lambda(lambda z: z[:, row_idx[0]:row_idx[1], col_idx[0]:col_idx[1], :])(inputs)
            list_patch.append(x_patch)

    x = [model(patch) for patch in list_patch]
    outputs = Average()(x)
    model = Model(inputs=inputs, outputs=outputs, name='Discriminator')
    return model


def generator_containing_discriminator(generator, discriminator):
    inputs = Input(shape=image_shape)
    generated_image = generator(inputs)
    outputs = discriminator(generated_image)
    model = Model(inputs=inputs, outputs=outputs)
    return model


if __name__ == '__main__':
    g = generator_model()
    g.summary()
    d = discriminator_model()
    d.summary()
    plot_model(d)
    m = generator_containing_discriminator(generator_model(), discriminator_model())
    m.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 1 2800        input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, None, None, 1 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, None, None, 1 10100       leaky_re_lu_1[0][0]              
__________________________________________________________________________________________________
batch_norm

In [15]:
import glob as gb

import numpy as np
from PIL import Image

#import data_utils
#from losses import adversarial_loss, generator_loss
#from model import generator_model, discriminator_model, generator_containing_discriminator


def train(batch_size, epoch_num):
    # Note the x(blur) in the second, the y(full) in the first
    y_train, x_train = load_data(data_type='traindata')

    # GAN
    g = generator_model()
    d = discriminator_model()
    d_on_g = generator_containing_discriminator(g, d)

    # compile the models, use default optimizer parameters
    # generator use adversarial loss
    g.compile(optimizer='adam', loss=generator_loss)
    # discriminator use binary cross entropy loss
    d.compile(optimizer='adam', loss='binary_crossentropy')
    # adversarial net use adversarial loss
    d_on_g.compile(optimizer='adam', loss=adversarial_loss)

    for epoch in range(epoch_num):
        print('epoch: ', epoch + 1, '/', epoch_num)
        print('batches: ', int(x_train.shape[0] / batch_size))

        for index in range(int(x_train.shape[0] / batch_size)):
            # select a batch data
            image_blur_batch = x_train[index * batch_size:(index + 1) * batch_size]
            image_full_batch = y_train[index * batch_size:(index + 1) * batch_size]
            generated_images = g.predict(x=image_blur_batch, batch_size=batch_size)

            # output generated images for each 30 iters
            if (index % 3 == 0) and (index != 0):
                  generate_image(image_full_batch, image_blur_batch, generated_images,
                                          'result/interim/', epoch, index)

            # concatenate the full and generated images,
            # the full images at top, the generated images at bottom
            x = np.concatenate((image_full_batch, generated_images))

            # generate labels for the full and generated images
            y = [1] * batch_size + [0] * batch_size

            # train discriminator
            d_loss = d.train_on_batch(x, y)
            print('batch %d d_loss : %f' % (index + 1, d_loss))

            # let discriminator can't be trained
            d.trainable = False      #change here

            # train adversarial net
            d_on_g_loss = d_on_g.train_on_batch(image_blur_batch, [1] * batch_size)
            print('batch %d d_on_g_loss : %f' % (index + 1, d_on_g_loss))

            # train generator
            g_loss = g.train_on_batch(image_blur_batch, image_full_batch)
            print('batch %d g_loss : %f' % (index + 1, g_loss))

            # let discriminator can be trained
            d.trainable = True

            # output weights for generator and discriminator each 30 iters
            if (index % 3 == 0) and (index != 0):
                g.save_weights('weight/generator_weights.h5', True)
                d.save_weights('weight/discriminator_weights.h5', True)


def test(batch_size):
    # Note the x(blur) in the second, the y(full) in the first
    y_test, x_test = load_data(data_type='testdata')
    g = generator_model()
    g.load_weights('weight/generator_weights.h5')
    generated_images = g.predict(x=x_test, batch_size=batch_size)
    generate_image(y_test, x_test, generated_images, 'result/finally/')


def test_pictures(batch_size):
    data_path = '/content/ImageDeblurring/data/testdata/*.jpeg'
    images_path = gb.glob(data_path)
    data_blur = []
    for image_path in images_path:
        image_blur = Image.open(image_path)
        data_blur.append(np.array(image_blur))

    data_blur = np.array(data_blur).astype(np.float32)
    data_blur = normalization(data_blur)

    g = generator_model()
    g.load_weights('weight/generator_weights.h5')
    generated_images = g.predict(x=data_blur, batch_size=batch_size)
    generated = generated_images * 127.5 + 127.5
    for i in range(generated.shape[0]):
        image_generated = generated[i, :, :, :]
        Image.fromarray(image_generated.astype(np.uint8)).save('result/test/' + str(i) + '.png')


if __name__ == '__main__':
    train(batch_size=10, epoch_num=1)
    test(10)
    test_pictures(2)


epoch:  1 / 1
batches:  60
batch 1 d_loss : 0.629318


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


batch 1 d_on_g_loss : 0.520445
batch 1 g_loss : 15909.912109
batch 2 d_loss : 0.519690
batch 2 d_on_g_loss : 0.299375
batch 2 g_loss : 12021.034180
batch 3 d_loss : 0.763788
batch 3 d_on_g_loss : 0.382001
batch 3 g_loss : 12456.730469
batch 4 d_loss : 0.658212
batch 4 d_on_g_loss : 0.554184
batch 4 g_loss : 11818.064453
batch 5 d_loss : 0.448695
batch 5 d_on_g_loss : 0.105389
batch 5 g_loss : 11087.657227
batch 6 d_loss : 0.334821
batch 6 d_on_g_loss : 0.328490
batch 6 g_loss : 10245.420898
batch 7 d_loss : 0.318368
batch 7 d_on_g_loss : 0.320592
batch 7 g_loss : 9374.219727
batch 8 d_loss : 0.234936
batch 8 d_on_g_loss : 0.351288
batch 8 g_loss : 8424.071289
batch 9 d_loss : 0.095013
batch 9 d_on_g_loss : 0.196853
batch 9 g_loss : 8893.067383
batch 10 d_loss : 0.116044
batch 10 d_on_g_loss : 0.584265
batch 10 g_loss : 9565.675781
batch 11 d_loss : 0.176972
batch 11 d_on_g_loss : 0.119045
batch 11 g_loss : 7816.370117
batch 12 d_loss : 0.052456
batch 12 d_on_g_loss : 0.054786
batch 12 

batch 28 g_loss : 6184.250977
batch 29 d_loss : 0.046018
batch 29 d_on_g_loss : 0.258204
batch 29 g_loss : 7006.880371
batch 30 d_loss : 0.295024
batch 30 d_on_g_loss : 0.007033
batch 30 g_loss : 6615.312500
batch 31 d_loss : 0.058829
batch 31 d_on_g_loss : 0.026254
batch 31 g_loss : 5779.581055
batch 32 d_loss : 0.138609
batch 32 d_on_g_loss : 0.007409
batch 32 g_loss : 5232.753906
batch 33 d_loss : 0.260315
batch 33 d_on_g_loss : 0.016295
batch 33 g_loss : 5354.835449
batch 34 d_loss : 0.376455
batch 34 d_on_g_loss : 0.028436
batch 34 g_loss : 4363.938477
batch 35 d_loss : 0.832246
batch 35 d_on_g_loss : 0.016399
batch 35 g_loss : 5568.643066
batch 36 d_loss : 0.379841
batch 36 d_on_g_loss : 0.273492
batch 36 g_loss : 5525.744141
batch 37 d_loss : 0.358671
batch 37 d_on_g_loss : 0.229132
batch 37 g_loss : 5380.229492
batch 38 d_loss : 0.350960
batch 38 d_on_g_loss : 0.060484
batch 38 g_loss : 5670.647461
batch 39 d_loss : 0.305528
batch 39 d_on_g_loss : 0.145811
batch 39 g_loss : 562

batch 55 g_loss : 4751.438965
batch 56 d_loss : 0.604372
batch 56 d_on_g_loss : 0.069115
batch 56 g_loss : 5000.798340
batch 57 d_loss : 0.552567
batch 57 d_on_g_loss : 0.129236
batch 57 g_loss : 4426.183594
batch 58 d_loss : 0.513010
batch 58 d_on_g_loss : 0.171305
batch 58 g_loss : 4287.713867
batch 59 d_loss : 0.503296
batch 59 d_on_g_loss : 0.076203
batch 59 g_loss : 3381.321045
batch 60 d_loss : 0.588450
batch 60 d_on_g_loss : 0.405817
batch 60 g_loss : 3093.392090


In [16]:
from google.colab import files

#dir = os.chdir('/content/result/finally')

#!tar -cvzf testresult.tgz /content/result/test
!ls -l /content/result/test
#!ls -l   /content/ImageDeblurring/data/    
#for file in os.listdir(dir):
#files.download('testresult.tgz')

total 32
-rw-r--r-- 1 root root 13598 Jun  6 07:11 0.png
-rw-r--r-- 1 root root 11415 Jun  6 07:11 1.png
-rw-r--r-- 1 root root  2096 Jun  6 07:11 2.png
